In [24]:
#reference: https://github.com/lucktroy/DeepST
from __future__ import print_function
from keras.layers import (
    Input,
    Activation,
    merge,
    Dense,
    Reshape
)
from keras.layers.convolutional import Convolution2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from keras import backend as K
import numpy as np
from keras import metrics

In [25]:
def mean_squared_error(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true))

def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred) ** 0.5

In [26]:
#function for forming resnet links
def _shortcut(input, residual):
    return merge([input, residual], mode='sum')

In [27]:
#this implements operation of resnet modelv2: batch norm -> activation -> convolution
def _bn_relu_conv(nb_filter, nb_row, nb_col, subsample=(1, 1), bn=False):
    def f(input):
        if bn:
            input = BatchNormalization(mode=0, axis=1)(input)
        activation = Activation('relu')(input)
        return Convolution2D(nb_filter=nb_filter, nb_row=nb_row, nb_col=nb_col, subsample=subsample, border_mode="same")(activation)
    return f

In [28]:
#defining a single residual unit -> two operations (batch->act->conv) followed by residual (shortcut) linking
def _residual_unit(nb_filter, init_subsample=(1, 1)):
    def f(input):
        residual = _bn_relu_conv(nb_filter, 3, 3)(input)
        residual = _bn_relu_conv(nb_filter, 3, 3)(residual)
        return _shortcut(input, residual)
    return f

In [29]:
#defining a residual block of residual units repeated k times
def ResUnits(residual_unit, nb_filter, repetations=1):
    def f(input):
        for i in range(repetations):
            init_subsample = (1, 1)
            input = residual_unit(nb_filter=nb_filter,
                                  init_subsample=init_subsample)(input)
        return input
    return f

In [30]:
def stresnet(c_conf=(3, 2, 32, 32), p_conf=(3, 2, 32, 32), t_conf=(3, 2, 32, 32), nb_residual_unit=3):
    '''
    C - Temporal Closeness
    P - Period
    T - Trend
    configuration_format = (len_seq, channels, map_height, map_width)
    '''

    # main input
    main_inputs = []
    outputs = []
    #calling residual network for each temporal sequence of TEC maps
    for conf in [c_conf, p_conf, t_conf]:
        if conf is not None:
            len_seq, channels, map_height, map_width = conf
            #converting to a 3D matrix for future convolutions
            input = Input(shape=(channels * len_seq, map_height, map_width))
            main_inputs.append(input)
            # Conv1
            conv1 = Convolution2D(
                nb_filter=64, nb_row=3, nb_col=3, border_mode="same")(input)
            # [nb_residual_unit] Residual Units
            residual_output = ResUnits(_residual_unit, nb_filter=64,
                              repetations=nb_residual_unit)(conv1)
            # Conv2
            activation = Activation('relu')(residual_output)
            conv2 = Convolution2D(
                nb_filter=channels, nb_row=3, nb_col=3, border_mode="same")(activation)
            outputs.append(conv2)

    #fusing outputs of each resnet model through linear combination
    from iLayer import iLayer
    new_outputs = []
    for output in outputs:
        new_outputs.append(iLayer()(output))
    main_output = merge(new_outputs, mode='sum')

    # fusing with external component
    #TODO

    main_output = Activation('tanh')(main_output)
    model = Model(input=main_inputs, output=main_output)

    return model

In [31]:
np.random.seed(1337)
nb_epoch = 10  # number of epoch at training stage
nb_epoch_cont = 10  # number of epoch at training (cont) stage
batch_size = 2  # batch size

lr = 0.0002  # learning rate
#in a lookback window of 1 day:
#closeness will sample 12 maps every 15 mins for the previous 3 hours
#period will sample 24 maps every hour for the previous day
#trend will sample 8 maps every 3 hours for the previous day
len_closeness = 8  # length of closeness dependent sequence
len_period = 8  # length of peroid dependent sequence
len_trend = 8  # length of trend dependent sequence
nb_residual_unit = 4   # number of residual units
#operating in grayscale
channels = 1
map_height = 5
map_width = 5
c_conf = (len_closeness, channels, map_height,
          map_width) if len_closeness > 0 else None
p_conf = (len_period, channels, map_height,
          map_width) if len_period > 0 else None
t_conf = (len_trend, channels, map_height,
          map_width) if len_trend > 0 else None

In [33]:

print("loading data...")
#X_train, Y_train, X_test, Y_test = load_data()

#f2 = open('tec_maps.pkl', 'r')
#tec_maps = pickle.load(f2)
#f2.close()

close_maps = np.random.rand(10,8,5,5)
period_maps = np.random.rand(10,8,5,5)
trend_maps = np.random.rand(10,8,5,5)
Y_train = np.random.rand(10,1,5,5)

X_train = [close_maps, period_maps,trend_maps]

model = stresnet(c_conf, p_conf, t_conf, nb_residual_unit=1)
adam = Adam(lr=lr)
model.compile(loss='mse', optimizer=adam, metrics=[rmse])
print (model.summary())

model.fit(X_train, Y_train,
            nb_epoch=nb_epoch,
            batch_size=batch_size,
            validation_split=0.1,
            verbose=1)

model.evaluate(X_test, Y_test, batch_size=Y_test.shape[0], verbose=0)

loading data...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_31 (InputLayer)           (None, 8, 5, 5)      0                                            
__________________________________________________________________________________________________
input_32 (InputLayer)           (None, 8, 5, 5)      0                                            
__________________________________________________________________________________________________
input_33 (InputLayer)           (None, 8, 5, 5)      0                                            
__________________________________________________________________________________________________
conv2d_301 (Conv2D)             (None, 8, 5, 64)     2944        input_31[0][0]                   
_____________________________________________________________________________________________

ValueError: Error when checking target: expected activation_287 to have shape (8, 5, 1) but got array with shape (1, 5, 5)